## Importing

In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np

from LibSlicerUnityConnection.UnityConnMsgEncoderInterface import *
from LibSlicerUnityConnection.CoreServer import *
from LibSlicerUnityConnection.VertexMeshRender import VertexMeshRender
from LibSlicerUnityConnection.WebVertexInfoServiceZMQ import WebVertexInfoServiceZMQ
import slicer
import vtk.util.numpy_support as ns
import copy

In [ ]:
connServices = SlicerUnityConnServiceNode()
localDataNode = SlicerUnityConnLocalDataNode()

## Function

In [ ]:
mainUnityController = SlicerUnityMainControllerZMQ("localhost",32000)

### Mesh

In [ ]:
import open3d as o3d
inputModelNode = slicer.util.getNode('node')
polydata = inputModelNode.GetPolyData()
points = polydata.GetPoints()
numPoints = points.GetNumberOfPoints()
colors = inputModelNode.GetDisplayNode().GetOutputPolyData().GetPointData().GetScalars()
if colors is None:
    ambient_color = inputModelNode.GetDisplayNode().GetColor()
    ambient_color_np = np.array(ambient_color)
    color_array = (ambient_color_np.reshape([1,3]).repeat(numPoints,axis=0)*255).astype("uint8")
else:
    color_array = ns.vtk_to_numpy(colors)

faces = polydata.GetPolys()
vertex_array = ns.vtk_to_numpy(points.GetData()).astype("float32")
face_array = ns.vtk_to_numpy(faces.GetData()).astype("int32")
face_array = face_array.reshape(-1, face_array[0] + 1)[::, 1:]

color_array = color_array.astype("float32") / 255
color_rgba = np.zeros([numPoints,4],dtype='float32')
color_rgba[:,0:3] = color_array[:,:]
color_rgba[:,3] = 1.0

modelMesh = o3d.geometry.TriangleMesh()
modelMesh.vertices = o3d.utility.Vector3dVector(vertex_array)
modelMesh.triangles = o3d.utility.Vector3iVector(face_array)
modelMesh.compute_vertex_normals()
normal_array = np.asarray(modelMesh.vertex_normals).astype("float32")

vertexMeshRender = VertexMeshRender("newNode")
vertexMeshRender.Initiate(vertex_array, face_array, color_array)

In [ ]:
# create model
msg_CREATEMODEL = {
    "vertexs" : vertex_array,
    "faces" : face_array,
    "colors" : color_rgba,
    "normals" : normal_array,
    "transform" : np.diag([1,1,1,1]).astype("float32"),
    "scale" : np.array([1,1,1],dtype="float32"),
    "name" : "My_Model3",
    "attachTool" : -1
}

msgbyte_CREATEMODEL = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CREATE_MODEL)(msg_CREATEMODEL).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CREATE_MODEL,msgbyte_CREATEMODEL)

In [ ]:
# change model pose
trans = np.diag([1.,1.,1.,1.])
trans[0:3,3] = np.array([0,0,0])[:]
scale = np.array([1,1,1],dtype="float32")

msg_CHANGEMODELTRANS = {
    "name" : "My_Model3",
    "transform" : trans,
    "scale" : scale
}

msgbyte_CHANGEMODELTRANS = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_MODEL_TRANSFROM)(msg_CHANGEMODELTRANS).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_MODEL_TRANSFROM,msgbyte_CHANGEMODELTRANS)

In [ ]:
# delete model
msg_DELETEMODEL = {
    "name" : "My_Model3"
}

msgbyte_DELETEMODEL = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.DELETE_MODEL)(msg_DELETEMODEL).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.DELETE_MODEL,msgbyte_DELETEMODEL)

In [ ]:
# change model visibility
msg_CHANGE_MODEL_VISIBILITY = {
    "name" : "My_Model3",
    "stat" : True
}

msgbyte_CHANGE_MODEL_VISIBILITY = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_MODEL_VISIBILITY)(msg_CHANGE_MODEL_VISIBILITY).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_MODEL_VISIBILITY,msgbyte_CHANGE_MODEL_VISIBILITY)

In [ ]:
# change model paintable
msg_CHANGE_MODEL_PAINTABLE_STAT = {
    "name" : "My_Model3",
    "stat" : False
}

msgbyte_CHANGE_MODEL_PAINTABLE_STAT = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_MODEL_PAINTABLE_STAT)(msg_CHANGE_MODEL_PAINTABLE_STAT).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_MODEL_PAINTABLE_STAT,msgbyte_CHANGE_MODEL_PAINTABLE_STAT)

In [ ]:
syncer = WebVertexInfoServiceZMQ(32130,b"headSurfaceVertexColor",20,"localhost")

def OnChangeColorsForMesh(colors):
    vertexMeshRender.ChangeVertexColor(colors)
    DoForceRender()

syncer.RegisCallBack("changeVetexColorFace",OnChangeColorsForMesh)

In [ ]:
# change model vertex publish stat
msg_CHANGE_PAINTABLE_MODEL_STREAM_STAT = {
    "name" : "My_Model3",
    "stat" : False,
    "port" : 32130
}

msgbyte_CHANGE_PAINTABLE_MODEL_STREAM_STAT = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_PAINTABLE_MODEL_STREAM_STAT)(msg_CHANGE_PAINTABLE_MODEL_STREAM_STAT).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_PAINTABLE_MODEL_STREAM_STAT,msgbyte_CHANGE_PAINTABLE_MODEL_STREAM_STAT)

In [ ]:
syncer.Stop()

In [ ]:
syncer.Start()

### Volumes

In [ ]:
connServices.volumeDataSyncerList["T1VolumeData"] = VolumeDataSenderMMAP("T1VolumeData")

In [ ]:
connServices.volumeDataSyncerList["T1VolumeData"].StreamVolume(
    slicer.util.getNode('xx')
)

In [ ]:
msg_REFRESH_VOLUME_DATA = {
    "metaDataLocate" : "T1VolumeData",
    "transform" : np.diag([1.,1.,1.,1.]),
    "scale" : np.array([1,1,1],dtype="float32"),
    "attachTool" : -1
}

msgbyte_REFRESH_VOLUME_DATA = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.REFRESH_VOLUME_DATA)(msg_REFRESH_VOLUME_DATA).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.REFRESH_VOLUME_DATA,msgbyte_REFRESH_VOLUME_DATA)

In [ ]:
from LibSlicerUnityConnection.VolumeDataSenderMMAP import *

In [ ]:
### RGB volume
rgbVol = VolumeDataSenderMMAP("rawFemale_r", mainVolumeType = MainVolumeType.RGB)

In [ ]:
rgbVol.StreamVolumeRGB(
    slicer.util.getNode('rawFemale_b'),
    slicer.util.getNode('rawFemale_g'),
    slicer.util.getNode('rawFemale_r')
)

In [ ]:
msg_REFRESH_VOLUME_DATA = {
    "metaDataLocate" : "rawFemale_r",
    "transform" : np.diag([1.,1.,1.,1.]),
    "scale" : np.array([1,1,1],dtype="float32"),
    "attachTool" : -1
}

msgbyte_REFRESH_VOLUME_DATA = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.REFRESH_VOLUME_DATA)(msg_REFRESH_VOLUME_DATA).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.REFRESH_VOLUME_DATA,msgbyte_REFRESH_VOLUME_DATA)

In [ ]:
msg_CHANGE_VOLUME_CUTOFF_BOX_NUM = {
    "metaDataLocate" : "females",
    "num" : 1
}

msgbyte_CHANGE_VOLUME_CUTOFF_BOX_NUM = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_VOLUME_CUTOFF_BOX_NUM)(msg_CHANGE_VOLUME_CUTOFF_BOX_NUM).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_VOLUME_CUTOFF_BOX_NUM,msgbyte_CHANGE_VOLUME_CUTOFF_BOX_NUM)


In [ ]:
from LibSlicerUnityConnection.VolumeRenderController import *

In [ ]:
msg_UPDATE_VOLUME_RENDER_MODE = {
    "metaDataLocate": "females",
    "mode": VolumeRenderMode.DirectVolumeRendering
}

msgbyte_UPDATE_VOLUME_RENDER_MODE = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.UPDATE_VOLUME_RENDER_MODE)(msg_UPDATE_VOLUME_RENDER_MODE).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.UPDATE_VOLUME_RENDER_MODE,msgbyte_UPDATE_VOLUME_RENDER_MODE)

In [ ]:
msg_CHANGE_VOLUME_PAINTABLE_STAT = {
    "metaDataLocate" : "T1VolumeData",
    "stat" : False
}

msgbyte_CHANGE_VOLUME_PAINTABLE_STAT = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_VOLUME_PAINTABLE_STAT)(msg_CHANGE_VOLUME_PAINTABLE_STAT).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_VOLUME_PAINTABLE_STAT,msgbyte_CHANGE_VOLUME_PAINTABLE_STAT)

In [ ]:
msg_CHANGE_VOLUME_MASK_DISPLAY_STAT = {
    "metaDataLocate" : "T1VolumeData",
    "stat" : True
}

msgbyte_CHANGE_VOLUME_MASK_DISPLAY_STAT = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_VOLUME_MASK_DISPLAY_STAT)(msg_CHANGE_VOLUME_MASK_DISPLAY_STAT).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_VOLUME_MASK_DISPLAY_STAT,msgbyte_CHANGE_VOLUME_MASK_DISPLAY_STAT)

In [ ]:
msg_CHANGE_VOLUME_INTERACT_COLOR = {
    "metaDataLocate" : "T1VolumeData",
    "colorId" : 1
}

msgbyte_CHANGE_VOLUME_INTERACT_COLOR = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_VOLUME_INTERACT_COLOR)(msg_CHANGE_VOLUME_INTERACT_COLOR).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_VOLUME_INTERACT_COLOR,msgbyte_CHANGE_VOLUME_INTERACT_COLOR)

In [ ]:
msg_FORCE_CLEAN_VOLUME_INTERACT_MASK = {
    "metaDataLocate" : "T1VolumeData",
}

msgbyte_FORCE_CLEAN_VOLUME_INTERACT_MASK = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.FORCE_CLEAN_VOLUME_INTERACT_MASK)(msg_FORCE_CLEAN_VOLUME_INTERACT_MASK).getEncodedData()

connServices.mainUnityController.RequireChange(SlicerUnityCommandType.FORCE_CLEAN_VOLUME_INTERACT_MASK,msgbyte_FORCE_CLEAN_VOLUME_INTERACT_MASK)

In [ ]:
msg_CHANGE_VOLUME_VISIBILITY = {
    "metaDataLocate" : "T1VolumeData",
    "stat" : True
}

msgbyte_CHANGE_VOLUME_VISIBILITY = SlicerUnityCommandEncoderFactory().getEncoder(SlicerUnityCommandType.CHANGE_VOLUME_VISIBILITY)(msg_CHANGE_VOLUME_VISIBILITY).getEncodedData()

mainUnityController.RequireChange(SlicerUnityCommandType.CHANGE_VOLUME_VISIBILITY,msgbyte_CHANGE_VOLUME_VISIBILITY)